## Individual Proposal

#### Broad Question
Question 2: We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.

#### Specific Question
Does the background of a player (Age, Experience) predictd how long they are on the server? 

This question addresses the overall question as if people play for longer, they will contribute more data for the experiment.

### Data <br />

The data consists of 2 datasets, for a total of 9 variables (excluding id's such as name and hashed emails) which are: <br />

Dataset: sessions.csv
`start_time`: Start time and date formatted as `DD/MM/YYYY hh:mm`<br />
`end_time`: End time and date formatted as `DD/MM/YYYY hh:mm`<br />
`original_start_time`: Start time and date formatted in UNIX (ms)<br />
`original_end_time`: End time and date formatted in UNIX (ms)<br />

Dataset: players.csv
`experience`: Experience level of player as a string that can be a value of :`Pro`, `Veteran`, `Regular`, `Amateur`, `Beginner`.<br />
`subscribe`: Is the person subscribed to the newsletter? String value of either `True` or `False`<br />
`played_hours`: How many hours the person has played, formatted as numeric float.<br />
`gender`: Gender of respondent, String value. <br />
`Age`: Age of respondent, numeric float <br />

### Plan

We will use the players dataset to see if the background of the player can be used to predict how long they would play. This is useful as if more people play for longer, than more licenses will be needed. In order to achieve this, we will predict using 
k nearest neighbours regression. This is a suitable method as we are predicting a contious variable rather than a discrete one, so we cannot use classificaiton methods. Since we don't know the relationship, and we are also using experience in a numeric form, we don't know if the resulting relationship will be linear. Thus, a knn regression will probably yield more accurate results. A key limitation of this is that we will need to factor in discrete data (experience, gender) into this calculation, although there are ways to format it to make it viable. 

We will create a model by using age, experience (converted to numeric) as our independent variables, and the playtime as our dependent variable. We will then use k nearest neighbour regression to create a model to predict playtime of new players. We will split the data into a training and a testing dataset, so that we can validate our model after we create it. This split will likely be a 60/30 split. For non numeric data, we will convert each unique data value into a number. For experience, we will have beginner as 0.2, and move up in experience until we get pro as 1. This allows us to avoid normalization for this variable. We will have to normalize age to a value between 0 and 1. 

### Initial Data Wrangling and Summary Statistics


#### Libraries:

In [1]:
### Run this cell before continuing.
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)
options(repr.matrix.max.rows = 6)
source("cleanup.R")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection
